Duplicate Question Pair Detection in Quora Question Pairs 

Import the required libraries

In [52]:
!pip install nltk

You should consider upgrading via the '/Users/sarthak/.pyenv/versions/3.9.13/bin/python3.9 -m pip install --upgrade pip' command.


In [53]:
# importing required libraries for duplicquestion detection
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sarthak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


All Functions

In [54]:
def pun(text):
    text = re.sub(r'[^\w\s]','',text)
    return text

def clean_dataset(df):
    #drop na
    df.dropna(inplace=True)
    indices_to_keep=~df.isin([np.nan, np.inf, -np.inf]).any(1)
    df=df[indices_to_keep].astype(np.float64)
    return df

def length(text):
    return len(text)

def diff_length(text1, text2):
    return abs(len(text1) - len(text2))

def len_ratio(text1,text2):
    return float(len(text1)/len(text2))

def com_low(text):
    text1=text[0]
    text2=text[1]
    #split text1 and 2
    text1=text1.split()
    text2=text2.split()
    #check for lower case
    test1_low=set()
    test2_low=set()
    # check if the word is lower case or not
    for word in text1:
        if word.islower():
            test1_low.add(word)
    for word in text2:
        if word.islower():
            test2_low.add(word)
    #check for common lower case words
    common_low=test1_low.intersection(test2_low)
    return len(list(common_low))

def stop(text):
    text1=text[0]
    text2=text[1]
    #split text1 and 2
    text1=text1.split()
    text2=text2.split()
    #check for stop words
    text1_low=set()
    text2_low=set()
    stop_words=set(stopwords.words('english'))
    # check if the word is stop word or not
    for word in text1:
        #checkif word is lower case
        if word.islower():
            text1_low.add(word)
    
    text1=list(text1_low)
    for word in text2:
        #check if word is lower case
        if word.islower():
            text2_low.add(word)
    text2=list(text2_low)
    #check for lowercase as well as stop_words
    text1=[w for w in text1 if not w.lower() in stop_words]
    text2=[w for w in text2 if not w.lower() in stop_words]
    text1=set(text1)
    text2=set(text2)
    #check for common stop words
    common_stop=text1.intersection(text2)
    return len(list(common_stop))

def last(text):
    text1=text[0]
    text2=text[1]
    #split text1 and 2
    text1=list(text1.split(" "))
    text2=list(text2.split(" "))
    #check for last word
    if text1[len(list(text1))-1]==text2[len(list(text2))-1]:
        return 1
    else:
        return 0

def com_cap(text):
    text1=text[0]
    text2=text[1]
    #split text1 and 2
    text1=text1.split()
    text2=text2.split()
    #check for common capital words
    test1_cap=set()
    test2_cap=set()
    # check if the word is capital or not
    for word in text1:
        if word.isupper():
            test1_cap.add(word)
    for word in text2:
        if word.isupper():
            test2_cap.add(word)
    #check for common capital words
    common_cap=test1_cap.intersection(test2_cap)
    return len(list(common_cap))

def pre(text,n):
    text1=text[0]
    text2=text[1]
    #split text1 and 2
    text1=text1.split()
    text2=text2.split()
    #check for common prefix
    str1=text1
    str2=text2
    str1.sort()
    i=0
    j=0
    str2.sort()
    ans = 0
    while True:
        if i >= len(str1) or j >= len(str2):
            break
        if len(str1[i]) < n:
            i += 1
            continue
        if len(str2[j]) < n:
            j += 1
            continue
        s1 = str1[i][0:n]
        s2 = str2[j][0:n]
        if s1 == s2:
            ans += 1
            i += 1 
            j += 1 
        else:
            if str1[i]>str2[j]:
                j += 1
            else:
                i += 1
    return ans

def misc1(text):
    #check if not is present of not
    text=text.split()
    if 'not' in text:
        return 1
    else:
        return 0
    
def misc2(text):
    text1=text[0]
    text2=text[1]
    #split text1 and 2
    text1=text1.split()
    text2=text2.split()
    #check is not is present in both
    if 'not' in text1 and 'not' in text2:
        return 1
    else:
        return 0

def misc3(text):
    text1=text[0]
    text2=text[1]
    #split text1 and 2
    text1=text1.split()
    text2=text2.split()
    #check if word is in digit and is in text2 as well
    for word in text1:
        if word.isdigit():
            if word in text2:
                return 1
    return 0

def misc4(text):
    text1=text[0]
    text2=text[1]
    ps=PorterStemmer()
    #tokenize word1 and word2
    text1=text1.split()
    text2=text2.split()
    text_s1=[]
    text_s2=[]
    #stemming
    for word in text1:
        text_s1.append(ps.stem(word))
    for word in text2:
        text_s2.append(ps.stem(word))
    text1_low=set()
    text2_low=set()
    # check if the word is lower case or not
    for word in text_s1:
        if word.islower():
            text1_low.add(word)
    for word in text_s2:
        if word.islower():
            text2_low.add(word)
    #check for common lower case words
    common_low=text1_low.intersection(text2_low)
    return len(list(common_low))


Importing the dataset

In [55]:
data = pd.read_csv("./train.csv")
data = data.dropna(how="any").reset_index(drop=True)

In [56]:
#apply pun function
data['question1']=data['question1'].apply(pun)
data['question2']=data['question2'].apply(pun)
#apply lenght function
data['l1']=data['question1'].apply(length)
data['l2']=data['question2'].apply(length)
#apply diff_length function
data['l3']=abs(data['l1']-data['l2'])
#apply len_ratio function
data['l4']=(data['l1']/data['l2'])
#copy data in to data_l
data_l=data.copy()
#apply com_low function
data['lc1']=data[['question1','question2']].apply(com_low,axis=1)

sen1=data['question1'].str.len().idxmax()
sen2=data['question2'].str.len().idxmax()
#get the longest sentence
l1=len(data['question1'][sen1])
l2=len(data['question2'][sen2])
#get max between l1 and l2
l=max(l1,l2)
data['lc2']=data[['question1','question2']].apply(com_low,axis=1)
data['lc2']=data['lc2']/l
#copy data in to data_2
data_2=data.copy()
data['lcxs_1']=data[['question1','question2']].apply(lambda x:stop(x),axis=1)
data['lxcs_2']=(data['lcxs_1'])/l
#copy data in to data_3
data_3=data.copy()
data.head()

,id,qid1,qid2,question1,question2,is_duplicate,l1,l2,l3,l4,lc1,lc2,lcxs_1,lxcs_2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,65,56,9,1.160714,10,0.008850,5,0.004425
1,1,3,4,What is the story of Kohinoor KohiNoor Diamond,What would happen if the Indian government sto...,0,46,83,37,0.554217,1,0.000885,0,0.000000
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,72,58,14,1.241379,2,0.001770,1,0.000885
3,3,7,8,Why am I mentally very lonely How can I solve it,Find the remainder when math2324math is divide...,0,48,55,7,0.872727,0,0.000000,0,0.000000
4,4,9,10,Which one dissolve in water quikly sugar salt ...,Which fish would survive in salt water,0,73,38,35,1.921053,3,0.002655,2,0.001770


In [59]:
data['lw_1']=data[['question1','question2']].apply(last,axis=1)
data_4=data.copy()
data['cap_1']=data[['question1','question2']].apply(com_cap,axis=1)
data['cap_2']=data['cap_1']/l
data_5=data.copy()

#apply all the prefix function
data['pre_1']=data[['question1','question2']].apply(lambda x:pre(x,3),axis=1)
data['pre_3']=data[['question1','question2']].apply(lambda x:pre(x,4),axis=1)
data['pre_5']=data[['question1','question2']].apply(lambda x:pre(x,5),axis=1)
data['pre_7']=data[['question1','question2']].apply(lambda x:pre(x,6),axis=1)
data['pre_2']=data['pre_1']/l
data['pre_4']=data['pre_3']/l
data['pre_6']=data['pre_5']/l
data['pre_8']=data['pre_7']/l
data_6=data.copy()

#apply all the misc function
data['misc1'] = data['question1'].apply(misc1)
data['misc2'] = data['question2'].apply(misc1)

data['misc3'] = data[['question1','question2']].apply(misc2,axis=1)
data['misc4'] = data[['question1','question2']].apply(misc3,axis=1)
data['misc5']= data[['question1','question2']].apply(lambda x:misc4(x),axis=1)
data_7=data.copy()